# Dynamic Truncation + Validation + Cross-Encoder + Fusion

## Notebook Summary: Truncation + Validation Fusion RAG (Setup 8)

This notebook implements an optimized RAG pipeline for extractive QA over telecom documents, integrating robust safeguards against hallucination and prompt overload.

### Key Enhancements:

1. **Cross-Encoder Reranking**  
   Initial chunk retrieval is reranked using `cross-encoder/ms-marco-MiniLM-L-6-v2` for deep semantic alignment.

2. **Dynamic Truncation**  
   Top-ranked chunks are split into overlapping word windows (150 tokens, stride 75), and re-scored using a weighted combination of TF-IDF and lexical overlap. Only the most relevant spans are retained.

3. **Fusion Prompting**  
   Selected spans are fused into a single prompt, annotated with source identifiers, and passed to a LoRA-fine-tuned LLaMA-2 model for extractive generation.

4. **Fuzzy Validation**  
   Ensures that the final answer text is approximately contained within the retrieved spans, using token similarity. Warnings are issued if validation fails.

5. **Evaluation**  
   Assessed on 100 QA pairs with:
   - **SQuAD (Exact Match, F1)**
   - **ROUGE-L**
   - **BLEU**

This setup provides the most robust centralized RAG architecture so far — balancing precision, safety, and context richness — and is a strong candidate for downstream deployment or FL extension.

In [1]:
# Imports
import re
import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from difflib import SequenceMatcher

def is_similar(a, b, threshold=0.75):
    return SequenceMatcher(None, a, b).ratio() >= threshold

def truncate_and_filter_chunks(chunks, query, window_size=150, stride=75, max_windows=5):
    STOPWORDS = set(stopwords.words("english"))

    def normalize(text):
        return re.sub(r'\W+', ' ', text.lower())

    def lexical_overlap(query, span):
        q_tokens = set(normalize(query).split()) - STOPWORDS
        c_tokens = set(normalize(span).split()) - STOPWORDS
        return len(q_tokens & c_tokens) / (len(q_tokens | c_tokens) + 1e-5)

    def tfidf_score(query, span):
        vec = TfidfVectorizer().fit([query, span])
        X = vec.transform([query, span])
        return (X[0] @ X[1].T).A[0][0]

    scored_spans = []
    for chunk in chunks:
        words = chunk["content"].split()
        for i in range(0, len(words), stride):
            span_words = words[i:i + window_size]
            if len(span_words) < 30:
                continue
            span = " ".join(span_words)
            score = 0.6 * lexical_overlap(query, span) + 0.4 * tfidf_score(query, span)
            scored_spans.append({
                "content": span,
                "score": score,
                "source": chunk.get("source", "unknown")
            })

    return sorted(scored_spans, key=lambda x: x["score"], reverse=True)[:max_windows]

In [3]:
import torch
# Load FAISS index and chunked docs
index = faiss.read_index("/mnt/data/RAG/3gpp_index.faiss")
with open("/mnt/data/RAG/3gpp_chunks.pkl", "rb") as f:
    documents = pickle.load(f)

# Load embedding + cross-encoder models
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

# FAISS + Cross-Encoder Retrieval
def retrieve_with_rerank(query, top_k=5):
    query_vec = embedding_model.encode(query, normalize_embeddings=True)
    query_vec = np.array(query_vec).reshape(1, -1).astype("float32") 

    D, I = index.search(query_vec, top_k * 2)

    initial_results = [documents[i] for i in I[0]]
    pairs = [(query, doc["content"]) for doc in initial_results]

    scores = reranker.predict(pairs)
    reranked = sorted(zip(scores, initial_results), key=lambda x: x[0], reverse=True)[:top_k]

    return [doc for _, doc in reranked]

# Multi-Chunk Fusion Prompt Builder
SYSTEM_PROMPT = (
    "You are a precise assistant. Extract the exact answer span from the context. "
    "Do not paraphrase, summarize, or add extra information. "
    "The answer must appear exactly in the context. "
    "If the context lists multiple conditions, actions, or branches, include them all as written. "
    "Do not summarize or paraphrase — copy the exact text from the context, line by line."
)

def build_fusion_prompt(context_chunks, question):
    context_lines = []
    for chunk in context_chunks:
        source = chunk.get("source", "unknown").split("/")[-1]
        context_lines.append(f"[Source: {source}]\n-----\n{chunk['content'].strip()}")
    fused_context = "\n\n".join(context_lines)

    user_prompt = (
        f"Context:\n{fused_context}\n\n"
        f"Question: {question}\n"
        f"Answer from the context only:"
    )

    return f"<s>[INST] <<SYS>>\n{SYSTEM_PROMPT}\n<</SYS>>\n\n{user_prompt} [/INST]"

# Output Cleaning
def clean_prediction(raw_text):
    answer = raw_text.split("[/INST]")[-1].strip()
    answer = re.sub(r"[^\w\s\-.,:/()]", "", answer)
    answer = re.sub(r'(\b.+?:)(\s*\1)+', r'\1', answer)

    tokens = answer.split()
    for i in range(1, len(tokens) // 2):
        if tokens[:i] == tokens[i:2*i]:
            answer = " ".join(tokens[:i])
            break

    sentence_end = re.search(r'[.?!]', answer)
    if sentence_end:
        answer = answer[:sentence_end.end()]
    return answer.strip()

# Load Fine-Tuned LLaMA-2 + Pipeline
model_path = "/mnt/data/llama2_qa_lora_output5/final"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16).to("cuda")

qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

def answer_with_fusion_cross_rag_truncated(question, top_k=6, max_windows=5, verbose=False):
    # Step 1: Retrieve + rerank with cross-encoder
    initial_chunks = retrieve_with_rerank(question, top_k=top_k)

    # Step 2: Dynamic truncation filtering
    final_chunks = truncate_and_filter_chunks(initial_chunks, question, max_windows=max_windows)

    # Step 3: Build fusion prompt
    prompt = build_fusion_prompt(final_chunks, question)

    # Step 4: Run model
    output = qa_pipeline(
        prompt,
        max_new_tokens=160,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )[0]["generated_text"]

    answer = clean_prediction(output)

    # Step 5: Fuzzy validation
    all_context = " ".join([c["content"] for c in final_chunks])
    if not any(is_similar(answer.lower(), c["content"].lower()) for c in final_chunks):
        print("🚨 WARNING: Approximate match for answer not found in final context. Review answer relevance.")

    if verbose:
        print("📌 Prompt (truncated):\n", prompt[:500], "...\n")
        print("🧾 Raw Output:\n", output)
        print("✅ Final Answer:\n", answer)
        for i, chunk in enumerate(final_chunks):
            print(f"\n--- Context {i+1} ---\n{chunk['content'][:300]}...\n")

    return answer, final_chunks

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [4]:
import re
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import json
from tqdm import tqdm
from evaluate import load

# Load QA pairs
def load_qa_pairs(path):
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]

qa_pairs = load_qa_pairs("3gpp_qa_100_pairs.jsonl")

# Load metrics
squad_metric = load("squad")
rouge = load("rouge")
bleu = load("bleu")

bleu_predictions = []
bleu_references = []
results = []

for sample in tqdm(qa_pairs):
    question = sample["question"]
    reference = sample["answer"]

    try:
        prediction, _ = answer_with_fusion_cross_rag_truncated(question)
    except Exception as e:
        print(f"⚠️ Error on: {question}\n{e}")
        prediction = ""

    # Add to metrics
    squad_metric.add(
        prediction={"id": str(hash(question)), "prediction_text": prediction},
        reference={"id": str(hash(question)), "answers": {"text": [reference], "answer_start": [0]}}
    )
    rouge.add(prediction=prediction, reference=reference)
    bleu_predictions.append(prediction)
    bleu_references.append([reference])
    results.append({
        "question": question,
        "reference": reference,
        "prediction": prediction
    })

# Compute final scores
squad_scores = squad_metric.compute()
rouge_scores = rouge.compute()
bleu_score = bleu.compute(predictions=bleu_predictions, references=bleu_references)["bleu"]

# Print results
print("\n📊 Final Evaluation Results (Setup 8 — Truncation + Validation + Cross-Encoder + Fusion):")
print(f"Exact Match (EM): {squad_scores['exact_match']:.2f}")
print(f"F1 Score        : {squad_scores['f1']:.2f}")
print(f"ROUGE-L         : {rouge_scores['rougeL']:.4f}")
print(f"BLEU            : {bleu_score:.4f}")

  1%|▍                                          | 1/100 [00:07<12:25,  7.53s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🚨 WARNING: Approximate match for answer not found in final context. Review answer relevance.


  2%|▊                                          | 2/100 [00:08<05:42,  3.50s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🚨 WARNING: Approximate match for answer not found in final context. Review answer relevance.


  3%|█▎                                         | 3/100 [00:12<06:25,  3.97s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

🚨 WARNING: Approximate match for answer not found in final context. Review answer relevance.


 99%|█████████████████████████████████████████▌| 99/100 [08:29<00:05,  5.07s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🚨 WARNING: Approximate match for answer not found in final context. Review answer relevance.


100%|█████████████████████████████████████████| 100/100 [08:33<00:00,  5.13s/it]

🚨 WARNING: Approximate match for answer not found in final context. Review answer relevance.

📊 Final Evaluation Results (Setup 8 — Truncation + Validation + Cross-Encoder + Fusion):
Exact Match (EM): 2.00
F1 Score        : 21.19
ROUGE-L         : 0.2249
BLEU            : 0.0328
